# Ensemble methods

### Preprocess the time series data using Fast Fourier Transform

In [72]:
import numpy as np
from scipy.fftpack import fft
import os

XTrain = []
YTrain = []
#XTrain = np.loadtxt(os.getcwd() + r'/UCI HAR Dataset/UCI HAR Dataset/train/X_train.txt', delimiter=" ", dtype='str')
xTrainFileName = os.getcwd() + r'/UCI HAR Dataset/UCI HAR Dataset/train/X_train.txt'
with open(xTrainFileName, "r") as myfile:
    for line in myfile:
        XTrain.append(map(float, line.split()))
        
              
yTrainFileName = os.getcwd() + r'/UCI HAR Dataset/UCI HAR Dataset/train/y_train.txt'
with open(yTrainFileName, "r") as myfile:
    for line in myfile:
        YTrain.append(map(int, line.split()))

XTrain = np.asarray(XTrain)
YTrain = np.asarray(YTrain)

### Concatenate the acceleration for all the axes into one feature vector.

In [73]:
# The first 265 columns represent the acceleration for all the axes in the Time Series Domain
xf = fft(XTrain[:,0:265])

# timeSeriesFFT = []
# for row in xf:
#     timeSeriesFFT.append(row)

# print (np.array(timeSeriesFFT)).shape

newXTrain = np.append(XTrain[:,265:], xf , 1)
print newXTrain.shape

(7352, 561)


In [74]:
#Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier
def classifyRandomForestClassifier(XTrain, XTest, YTrain, YTest):
    clf = RandomForestClassifier(n_estimators=10)
    clf.fit(XTrain, YTrain)
    YPred = clf.predict(XTest)
    diff = YPred - YTest
    score = diff[diff == 0].size
    return (100.0 * score)/(YPred.size)

In [75]:
#K Nearest Neighbours Classifier
from sklearn.neighbors import KNeighborsClassifier
def classifyKNNClassifier(XTrain, XTest, YTrain, YTest):
    neigh = KNeighborsClassifier(n_neighbors=3)
    YPred = neigh.fit(XTrain, YTrain).predict(XTest)
    diff = YPred - YTest
    score = diff[diff == 0].size
    return (100.0 * score)/(YPred.size)

In [76]:
#Gaussian Naive Bayes Classifier
from sklearn.naive_bayes import GaussianNB
def classifyGaussianNaiveBayesClassifier(XTrain, XTest, YTrain, YTest):
    gnb = GaussianNB()
    YPred = gnb.fit(XTrain, YTrain).predict(XTest)
    diff = YPred - YTest
    score = diff[diff == 0].size
    return (100.0 * score)/(YPred.size)

In [77]:
#Multi Class SVM
from sklearn import svm
from sklearn.svm import NuSVC
def classifyMultiClassSVMClassifier(XTrain, XTest, YTrain, YTest):
    YPred = svm.SVC(kernel='linear').fit(XTrain, YTrain).predict(XTest)
    diff = YPred - YTest
    score = diff[diff == 0].size
    return (100.0 * score)/(YPred.size)

### Classify the signals into different activities. <br> </br> Use Leave-One-Out crossvalidation, where you test on one user and train on the rest of them.

In [ ]:
from sklearn import cross_validation
X = xf
y = YTrain
N = 7352
loo = cross_validation.LeaveOneOut(N)
print loo
y = y.ravel()

score = 0.0
for train_index, test_index in loo:
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    score += classifyRandomForestClassifier(X_train, X_test,  y_train, y_test)

score = score/N


### Implement a Python framework for mixture of experts, 
### where the user can assign multiple classifiers to a different region of the feature space.

## The results of these classifiers are forwarded to an additional classifier 
## (e.g. neural network) that is trained to optimally combine the output from the first stage.

## Graph of change of accuracy w.r.t. the number of used classifiers in the first stage.

## Is there any benefit of using mixture of experts on this dataset?

## Does the accuracy change if you use overlapping regions instead of disjunct ones?